# Loading results

In [ ]:
import DirectDmTargets as dddm

In [ ]:
import corner

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
import numpy as np

In [ ]:
import os

In [ ]:
import multihist as mh

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
from itertools import cycle
# import numpy as np
import scipy.optimize
# import matplotlib.pyplot as plt
# import seaborn as sns


In [ ]:
pd.set_option('display.width', 1000000)
pd.set_option('display.max_columns', 700)

In [ ]:
# loading normal results.
results = {}
for i in range(400):
    try:
        results[i] = dddm.load_nestle_samples(
            "../../results/nestle", i)
        print()
    except FileNotFoundError:
        pass

In [ ]:
for key in results.keys():
# for key in selected_keys:
    print(f"---{key}---")
    result = results[key]
    try:
        print('earth_shielding',result['config']['earth_shielding'])
#         if result['config']['earth_shielding'] == False:
#             print(result['config']['notes'])
#             pass
        dddm.nestle_corner(result, save = f"../results/nestle{key}/")
    except KeyError:
        pass
#     dddm.nestle_corner(result, save = f"../results/nestle{key}/")

In [ ]:
assert False

In [ ]:
def results_to_df(res):
    df = pd.DataFrame()
    items = sorted(list(res.keys()))
    df['item'] = items
    for key in res[np.min(list(res.keys()))].keys():
        if key in ['samples', 'weights']:
            continue
        if key == 'config' or key == 'res_dict':
            for sub_key in res[items[0]][key].keys():
                if sub_key == 'prior':
                    for sub_sub_key in res[items[0]][key][sub_key].keys():
                        if type(res[items[0]][key][sub_key][sub_sub_key]) == dict:
                            for sub_sub_sub_key in res[items[0]][key][sub_key][sub_sub_key].keys():
                                df[key+'_'+sub_key+'_'+sub_sub_key+'_'+sub_sub_sub_key] = [res[it][key][sub_key][sub_sub_key][sub_sub_sub_key] for it in items]
                        else:
                            df[key+'_'+sub_key+'_'+sub_sub_key] = [res[it][key][sub_key][sub_sub_key] for it in items]
                else:
                    df[key+'_'+sub_key] = [res[it][key][sub_key] for it in items]
            
        else:
            df[key] = [res[it][key] for it in items]
        
            
    df['mw'] = 10 ** df['config_mw']
    df['n_fit_parameters'] = [len(pars) for pars in df['config_fit_parameters']]
    return df
df = results_to_df(results)
df.head(5)

In [ ]:
mask = df['config_detector'] == 'Xe_migd'
mask2= df[mask]['res_dict_nestle_nposterior'] > 2000
selection = df[mask][mask2]['item'].values
df[mask][mask2]

In [ ]:
summary_cols = ['config_detector','mw', 'n_fit_parameters', 'res_dict_mass_fit_res']

In [ ]:
summary_cols

In [ ]:
mask = (
    (df['config_prior_density_mean'] == 0.40) & 
    (df['config_sigma'] == -45) 
    & (df['mw'] < 51)
#    & (df['mw'] < 51) & (df['mw'] > 49)
#    & (df['mw'] < 26) & (df['mw'] > 24)
#     (len(df['config_fit_parameters']) == 2)
)
df_sel =df[mask]
values = np.array([np.float(val.split('+/-')[0]) for val in df_sel['res_dict_mass_fit_res']])
errors = np.array([np.float(val.split('+/-')[1]) for val in df_sel['res_dict_mass_fit_res']])

df_sel['dmw_[%]'] = errors/values * 100 
if 'dmw_[%]' not in summary_cols:
    summary_cols += ['dmw_[%]']
res = {}
for sel in np.unique(df_sel['config_notes']):
    print(sel)
    sub_sel = df['config_notes'] == sel
    res[sel]=df_sel[sub_sel][summary_cols]


In [ ]:
df_sel[summary_cols].sort_values(['config_detector','mw'])


In [ ]:
results[0]['config']['prior']

In [ ]:
results[0]['res_dict'].keys()

In [ ]:
det = 'Ar'
mask = df['config_detector'] == det
sets = np.unique(df[mask]['config_notes'].values)
assert len(sets) == 2, "you have to manually select the keys since there are multiple sets in the query"
selected_keys_sets = [df[df['config_notes'] == _set]['item'].values for _set in sets] 
selected_keys_names = ["benchmark SHM constrained fit",
                      "benchmark SHM astr. unconstraint fit",
#                       "updated SHM constrained fit",
#                       "updated SHM astr. unconstraint fit"
                      ]
selected_keys_sets

In [ ]:
# def results_to_df(res):
#     df = pd.DataFrame()
#     items = sorted(list(res.keys()))
# #     df['item'] = np.arange(len(items))
#     for key_i, key in enumerate(items):
#         for subkey in res[key].keys():
#             print(key_i, key, subkey, res[key][subkey])
#             try: df.loc(key_i)[subkey] = res[key][subkey]
#             except:
#                 return df
# #         if key in ['samples', 'weights']:
# #             continue
# #         if True:
# #             for sub_key in res[items[0]][key].keys():
# # #                 if sub_key == 'prior':
# # #                     for sub_sub_key in res[items[0]][key][sub_key].keys():
# # #                         if type(res[items[0]][key][sub_key][sub_sub_key]) == dict:
# # #                             for sub_sub_sub_key in res[items[0]][key][sub_key][sub_sub_key].keys():
# # #                                 df[key+'_'+sub_key+'_'+sub_sub_key+'_'+sub_sub_sub_key] = [res[it][key][sub_key][sub_sub_key][sub_sub_sub_key] for it in items]
# # #                         else:
# # #                             df[key+'_'+sub_key+'_'+sub_sub_key] = [res[it][key][sub_key][sub_sub_key] for it in items]
# # #                 else:
# #                     df[key+'_'+sub_key] = [res[it][key][sub_key] for it in items]
            
# # #         else:
# #         df[key] = [res[it][key] for it in items]
        
            
#     df['mw'] = 10 ** df['config_mw']
#     df['n_fit_parameters'] = [len(pars) for pars in df['config_fit_parameters']]
#     return df
# test = results_to_df(dddm.convert_dic_to_savable(dddm.experiment))
# test.head(5)

In [ ]:
# for key in results.keys():
# # for key in selected_keys:
#     print(f"---{key}---")
#     result = results[key]
#     dddm.nestle_corner(result, save = f"../results/nestle_17nov{key}/")

In [ ]:
def get_posterior(samples, weights):
    # re-scale weights to have a maximum of one
    nweights = weights/np.max(weights)

    # get the probability of keeping a sample from the weights
    keepidx = np.where(np.random.rand(len(nweights)) < nweights)[0]
    # get the posterior samples
    return samples[keepidx,:]


In [ ]:
def bin_center(xedges, yedges):
    return 0.5 * (xedges[0:-1] + xedges[1:]), 0.5 * (yedges[0:-1] + yedges[1:])
    

In [ ]:
def get_hist(item, nbins = 45):
#     nbins = 45
    bin_range = [[1, 3], [-46, -44]]
    counts, xedges, yedges = np.histogram2d(*get_p_i(item), bins = nbins, range = bin_range)
    return counts , xedges, yedges

def get_hist_norm(item):
    counts , xedges, yedges = get_hist(item)
    return counts/np.sum(counts) , xedges, yedges

In [ ]:
# # fig = plt.figure(figsize=(7, 3))
# # ax = fig.add_subplot(131, title='imshow: square bins')
# from matplotlib.colors import LogNorm
# plt.imshow(test.T/np.sum(test.T), interpolation='nearest', origin='low',
#         extent=  [1, 3, -45.7, -44.3], norm=LogNorm())
# # plt.legend()
# plt.colorbar()

In [ ]:
def get_p_i(i):
    m, sig = get_posterior(results[i]['samples'], results[i]['weights']).T[:2]
    return np.array([m, sig])

In [ ]:
from matplotlib.colors import LogNorm
def combine_normalized(items):
    X, Y = np.meshgrid(*get_hist_norm(items[0])[1:])
    for i in items:
        c,_,_ = get_hist_norm(i)
        im = plt.pcolor(X,Y,c.T, norm=LogNorm(vmin = 1e-4,vmax = 1))  
    plt.colorbar()

In [ ]:
def pow10(x):
    return 10 ** x

In [ ]:
def confidence_plot(items, text_box = False):
    fig,ax=plt.subplots(figsize = (8,6))
    for k, item in enumerate(items):#, 78, 110 
        x,y =get_p_i(item)
        # Make a 2d normed histogram
        bin_range = [[0.5, 3], [-45.7, -44.3]]
        H,xedges,yedges=np.histogram2d(x,y,bins=500, range = bin_range, normed=True)

        norm=H.sum() # Find the norm of the sum
        # Set contour levels
        contour1=0.99
        contour2=0.95
        contour3=0.68
        # Take histogram bin membership as proportional to Likelihood
        # This is true when data comes from a Markovian process
        def objective(limit, target):
            w = np.where(H>limit)
            count = H[w]
            return count.sum() - target
        target1 = norm*contour1
        target2 = norm*contour2
        target3 = norm*contour3

        # Find levels by summing histogram to objective
        level1= scipy.optimize.bisect(objective, H.min(), H.max(), args=(target1,))
        level2= scipy.optimize.bisect(objective, H.min(), H.max(), args=(target2,))
        level3= scipy.optimize.bisect(objective, H.min(), H.max(), args=(target3,))

        levels=[level1, level2, level3,H.max()]
        # Pass levels to normed kde plot
        def av_levels(x):
            return [(x[i] + x[i+1])/2 for i in range(len(x)-1)]

        if levels[0]==levels[1]:
            print("ERRRRRRRRR\n\n")
            print(levels)
            levels[0] /= 1.01
            levels = np.unique(levels)
            print(levels)
        sns_ax = sns.kdeplot(x,y, shade=True,ax=ax,n_levels=levels,cmap="viridis",normed=True, 
                    cbar = False, vmin=levels[0], vmax=levels[-1])
        kwargs = {}
        if k is 0:
            kwargs['label'] = 'best fit'
        plt.scatter(np.mean(x),np.mean(y), c='black',
                    marker = '+',**kwargs)
        if k is 0:
            kwargs['label'] = 'benchmark value'
        plt.scatter(results[item]['config']['mw'],
                    results[item]['config']['sigma'], c='blue',
                    marker = 'x',
                    **kwargs)
        if k is 0:
            cbar = ax.figure.colorbar(sns_ax.collections[0])
            cbar.set_ticks(av_levels(np.linspace(0,1,4)))
            cbar.set_ticklabels(['$3\sigma$', '$2\sigma$', '$1\sigma$'])
            cbar.set_label("Posterior probability")
    secax = ax.secondary_xaxis('top', functions=(pow10, np.log10))
    x_ticks = [15, 25, 50, 100, 250, 500, 1000]
    for x_tick in x_ticks:
        ax.axvline(np.log10(x_tick), alpha = 0.1)
    secax.set_ticks(x_ticks)
    plt.xlim(np.log10(x_ticks[0]),np.log10(x_ticks[-1]))
    plt.xlabel("$\log_{10}(M_{\chi}$ $[GeV/c^{2}]$)")
    secax.set_xlabel("$M_{\chi}$ $[GeV/c^{2}]$")
    plt.ylabel("$\log_{10}(\sigma_{S.I.}$ $[cm^{2}]$)")
    plt.legend()

    if text_box:
        plt.text(0.05, 0.95, text_box, 
                 bbox=dict(facecolor="white",
                           boxstyle="round"), 
                 transform=ax.transAxes,
                 alpha=0.5)
#     plt.show()


In [ ]:
det = 'Xe_migd'
for i in selection:
    confidence_plot([results[i], text_box = f'{det}-detector')
#     if "realistic" in name:
#     plt.ylim(-49, -45)
#     else:
    plt.ylim(-45.7, -44.3)
    plt.title(f'{name} \n')
    plt.savefig(f"figures/{det}_{name.replace(' ', '-')}.png", dpi =300, bbox_inches="tight")
    plt.savefig(f"figures/{det}_{name.replace(' ', '-')}.pdf", dpi =300, bbox_inches="tight")
    plt.show()
                
#     break

In [ ]:
for i, name in enumerate(selected_keys_names):
    confidence_plot(selected_keys_sets[i], text_box = f'{det}-detector')
#     if "realistic" in name:
#     plt.ylim(-49, -45)
#     else:
    plt.ylim(-45.7, -44.3)
    plt.title(f'{name} \n')
    plt.savefig(f"figures/{det}_{name.replace(' ', '-')}.png", dpi =300, bbox_inches="tight")
    plt.savefig(f"figures/{det}_{name.replace(' ', '-')}.pdf", dpi =300, bbox_inches="tight")
    plt.show()
                
#     break

In [ ]:
det = 'Ge'
mask = df['config_detector'] == det
sets = np.unique(df[mask]['config_notes'].values)
assert len(sets) == 2, "you have to manually select the keys since there are multiple sets in the query"
selected_keys_sets = [df[df['config_notes'] == _set]['item'].values for _set in sets] 
selected_keys_names = ["benchmark SHM constrained fit",
                      "benchmark SHM astr. unconstraint fit",
#                       "updated SHM constrained fit",
#                       "updated SHM astr. unconstraint fit"
                      ]
selected_keys_sets

In [ ]:
for i, name in enumerate(selected_keys_names):
    confidence_plot(selected_keys_sets[i], text_box = f'{det}-detector')
    if "realistic" in name:
        plt.ylim(-49, -45)
    else:
        plt.ylim(-45.7, -44.3)
    plt.title(f'{name} \n')
    plt.savefig(f"figures/{det}_{name.replace(' ', '-')}.png", dpi =300, bbox_inches="tight")
    plt.savefig(f"figures/{det}_{name.replace(' ', '-')}.pdf", dpi =300, bbox_inches="tight")
    plt.show()
                

In [ ]:
det = 'Xe'
mask = df['config_detector'] == det
sets = np.unique(df[mask]['config_notes'].values)
assert len(sets) == 2, "you have to manually select the keys since there are multiple sets in the query"
selected_keys_sets = [df[df['config_notes'] == _set]['item'].values for _set in sets] 
selected_keys_names = ["benchmark SHM constrained fit",
                      "benchmark SHM astr. unconstraint fit",
#                       "updated SHM constrained fit",
#                       "updated SHM astr. unconstraint fit"
                      ]
selected_keys_sets

for i, name in enumerate(selected_keys_names):
    confidence_plot(selected_keys_sets[i], text_box = f'{det}-detector')
    if "realistic" in name:
        plt.ylim(-49, -45)
    else:
        plt.ylim(-45.7, -44.3)
    plt.title(f'{name} \n')
    plt.savefig(f"figures/{det}_{name.replace(' ', '-')}.png", dpi =300, bbox_inches="tight")
    plt.savefig(f"figures/{det}_{name.replace(' ', '-')}.pdf", dpi =300, bbox_inches="tight")
    plt.show()
                

In [ ]:
def get_quantile(item):
    _, xedges, _ = get_hist(item, nbins = 1000)
    data = get_p_i(item)
    results = np.zeros((len(xedges),3))
#     print(results)
    for i in range(len(xedges) - 1):
#         print(bin_i)
        mask = (xedges[i] < data[0]) & (data[0] < xedges[i+1])
        if np.sum(mask)>0:
#             print()
            results[i] =  0.5 * (xedges[i]+ xedges[i+1]), *np.percentile(data[1][mask], [5,95])
    return results[results[:,0] > 0]

In [ ]:
quantiles = get_quantile(0)
# quantiles = quantiles[quantiles]
plt.plot(quantiles[:,0],quantiles[:,1])
plt.plot(quantiles[:,0],quantiles[:,2])
# quantiles

In [ ]:
counts = get_hist(0)[0]
centers = bin_center(*get_hist(0)[1:])

In [ ]:
for count in counts.T:
    print(np.percentile(count, [5, 95]))

In [ ]:
combine_normalized([0])
quantiles = get_quantile(0)
# quantiles = quantiles[quantiles]
plt.plot(quantiles[:,0],quantiles[:,1])
plt.plot(quantiles[:,0],quantiles[:,2])

In [ ]:
np.shape(get_p_i(0)[:,])

In [ ]:
test = get_p_i(0)
np.array(test)[:,:100]

In [ ]:
confidence_plot(selected_keys)

In [ ]:
# import numpy as np
# import scipy.optimize
# import matplotlib.pyplot as plt
# import seaborn as sns

# # Generate some random data
# # x,y=np.random.randn(2,100000)
# fig,ax=plt.subplots(figsize = (8,6))
# for k, item in enumerate([74, 78, 110]):#, 78, 110 
#     x,y =get_p_i(item)
#     # Make a 2d normed histogram
#     H,xedges,yedges=np.histogram2d(x,y,bins=40,normed=True)

#     norm=H.sum() # Find the norm of the sum
#     # Set contour levels
#     contour1=0.99
#     contour2=0.95
#     contour3=0.68
#     # Take histogram bin membership as proportional to Likelihood
#     # This is true when data comes from a Markovian process
#     def objective(limit, target):
#         w = np.where(H>limit)
#         count = H[w]
#         return count.sum() - target
# #     contour={}
# #     target={}
# #     level={}
# #     sigmas = np.flip(range(2,4))
# #     for s in sigmas:
# #         contour[s] = sigma_to_p(s)
# #         print(contour[s])
# #         target[s]=norm*contour[s]
# #         level[s]=scipy.optimize.bisect(objective, H.min(), H.max(), args=(target[s],))
# #     # Set target levels as percentage of norm
#     target1 = norm*contour1
#     target2 = norm*contour2
#     target3 = norm*contour3



#     # Find levels by summing histogram to objective
#     level1= scipy.optimize.bisect(objective, H.min(), H.max(), args=(target1,))
#     level2= scipy.optimize.bisect(objective, H.min(), H.max(), args=(target2,))
#     level3= scipy.optimize.bisect(objective, H.min(), H.max(), args=(target3,))

#     # For nice contour shading with seaborn, define top level
# #     level[s+1]=H.max()
# #     levels=[level[s] for s in sigmas]
# #     levels.append(H.max())
#     levels=[level1, level2, level3,H.max()]
#     print(levels)
#     # Pass levels to normed kde plot
#     def av_levels(x):
#         return [(x[i] + x[i+1])/2 for i in range(len(x)-1)]
    
# #     sns_ax = sns.kdeplot(x,y, shade=True,ax=ax,n_levels=levels,cmap="viridis",normed=True, 
# #                 cbar = True, vmin=levels[0], vmax=levels[-1], 
# #                          cbar_kws={#"ticklocation":av_levels(levels),
# #                                    "ticks":[],
# #                                    #"format":">%s<"
# # #                                    "format":av_levels(levels)
# # #                                   "ticks":[1,3,4]
# #                                    "label":"significance"
# #                                   })
    
#     if levels[0]==levels[1]:
#         print("ERRRRRRRRR\n\n")
#         levels[0] /= 1.01
#     sns_ax = sns.kdeplot(x,y, shade=True,ax=ax,n_levels=levels,cmap="viridis",normed=True, 
#                 cbar = False, vmin=levels[0], vmax=levels[-1])
# #                          cbar_kws={#"ticklocation":av_levels(levels),
# #                                    "ticks":[],
# #                                    #"format":">%s<"
# # #                                    "format":av_levels(levels)
# # #                                   "ticks":[1,3,4]
# #                                    "label":"significance"
# #                                   })
#                             #          "labels":levels})
# #     fig.colorbar(cax,ticks=[-1, 0, 1])
# #     ax.set_aspect('equal')
#     kwargs = {}
#     if k is 0:
#         kwargs['label'] = 'best fit'
#     plt.scatter(np.mean(x),np.mean(y), c='black',
#                 marker = '+',**kwargs)
#     if k is 0:
#         kwargs['label'] = 'benchmark value'
#     plt.scatter(results[item]['config']['mw'],
#                 results[item]['config']['sigma'], c='blue',
#                 marker = 'x',
#                 **kwargs)
#     if k is 0:
#         cbar = ax.figure.colorbar(sns_ax.collections[0])
#     #     cbar = sns_ax.figure.colorbar()#[0].colorbar
#     #     print(av_levels(levels))
#     #     cbar.set_ticks(av_levels(levels))
#         cbar.set_ticks(av_levels(np.linspace(0,1,4)))
#     #     print(level1)
#         cbar.set_ticklabels(['$3\sigma$', '$2\sigma$', '$1\sigma$'])
#         cbar.set_label("Confidence interval")
#     #     cbar.set_nbins(4)
# secax = ax.secondary_xaxis('top', functions=(pow10, np.log10))
# # ax1.set_xticklabels([1])
# # ax.set_xlim(np.log10(10),np.log10(1000))
# x_ticks = [15, 25, 50, 100, 250, 500, 1000]
# for x_tick in x_ticks:
#     ax.axvline(np.log10(x_tick), alpha = 0.1)
# secax.set_ticks(x_ticks)
# plt.xlim(np.log10(x_ticks[0]),np.log10(x_ticks[-1]))
# plt.xlabel("$\log_{10}(M_{\chi}$ $[GeV/c^{2}]$)")
# secax.set_xlabel("$M_{\chi}$ $[GeV/c^{2}]$")
# plt.ylabel("$\log_{10}(\sigma_{S.I.}$ $[cm^{2}]$)")
# plt.legend()
# plt.show()


In [ ]:
from scipy.special import erf
def sigma_to_p(x):
    return erf(x/np.sqrt(2))

for x in range(1,5):
    print(x, sigma_to_p(x))

In [ ]:
combine_normalized(selection[0:2])

In [ ]:
combine_normalized(selected_keys_sets[1])

In [ ]:
np.percentile(np.linspace(0,100,100), [5,95])

In [ ]:
items = selected_keys[:3]
def get_df(items):
#     df = pd.DataFrame()
    for j, item in enumerate(items):
        posteriors = get_p_i(item)
        test_df = pd.DataFrame()
        
        test_df['$\log(M_\chi)$'] = posteriors[0]
        test_df['$\log(\sigma)$'] = posteriors[1]
        test_df['mass'] = 10**results[item]['config']['mw']
#         sns.jointplot(x="x", y="y", data=test_df, kind="kde",
#                      ylim = [-46,-44],
#                      xlim = [1,3]);
        if j == 0:
            df = test_df
        else:
            df = pd.concat([df, test_df])
#     df = df.assign(industry='yyy')
    return df
data = get_df(items)
print(np.unique(data.mass), len(data))
data.head(3)

In [ ]:
def plot_set(items, save_as):
    # data = sns.load_dataset('iris')
    # plt.figure(figsize=(15,10))
    data = get_df(items)
    print(np.unique(data.mass), len(data))
    def make_kde(*args, **kwargs):  
        #"cbar":True, 
    #     opts ={"n_levels":5}
        sns.kdeplot(n_levels = 3, *args, cmap=next(make_kde.cmap_cycle), **kwargs)

    def make_kde_cbar(*args, **kwargs):  
        #"cbar":True, 
    #     opts ={"n_levels":5}
        sns.kdeplot(n_levels = 3, cbar = True, *args, cmap=next(make_kde.cmap_cycle), **kwargs)
    kwargs ={"cbar":True, "n_levels":5}
    make_kde.cmap_cycle = cycle(('Blues_r', 'Oranges_r', 'Greens_r'))

    pg = sns.PairGrid(data, vars=('$\log(M_\chi)$', '$\log(\sigma)$'), hue='mass')
    pg.map_diag(sns.kdeplot)#, color = ({)'b', 'g', 'r'})
    pg.map_lower(make_kde)
    # pg.map_upper(make_kde_cbar)
    pg.fig.set_size_inches(15,10)
    pg.fig.savefig(f"{det}_{save_as}.png", dpi =300)
    pg.fig.savefig(f"{det}_{save_as}.pdf", dpi =300)
    plt.title(f"{save_as}")
    plt.show()
  

In [ ]:
for k in range(len(selected_keys_names)):
    plot_set(selected_keys_sets[k], selected_keys_names[k])

In [ ]:
# items = selected_keys[:3]
def get_df(items):
#     df = pd.DataFrame()
    for j, item in enumerate(items):
        posteriors = get_p_i(item)
        test_df = pd.DataFrame()
        
        test_df['$\log(M_\chi)$'] = posteriors[0]
        test_df['$\log(\sigma)$'] = posteriors[1]
        test_df['mass'] = 10**results[item]['config']['mw']
#         sns.jointplot(x="x", y="y", data=test_df, kind="kde",
#                      ylim = [-46,-44],
#                      xlim = [1,3]);
        if j == 0:
            df = test_df
        else:
            df = pd.concat([df, test_df])
#     df = df.assign(industry='yyy')
    return df

In [ ]:
items = selected_keys_sets[0]
data = get_df(items)
print(np.unique(data.mass), len(data))
data.head(3)

In [ ]:
%%time
# data = sns.load_dataset('iris')
# plt.figure(figsize=(15,10))
def make_kde(*args, **kwargs):  
    #"cbar":True, 
#     opts ={"n_levels":5}
    sns.kdeplot(n_levels = 3, *args, cmap=next(make_kde.cmap_cycle), **kwargs)

def make_kde_cbar(*args, **kwargs):  
    #"cbar":True, 
#     opts ={"n_levels":5}
    sns.kdeplot(n_levels = 3, cbar = True, *args, cmap=next(make_kde.cmap_cycle), **kwargs)
kwargs ={"cbar":True, "n_levels":5}
make_kde.cmap_cycle = cycle(('Blues_r', 'Oranges_r', 'Greens_r'))

pg = sns.PairGrid(data, vars=('$\log(M_\chi)$', '$\log(\sigma)$'), hue='mass')
pg.map_diag(sns.kdeplot)#, color = ({)'b', 'g', 'r'})
pg.map_lower(make_kde)
# pg.map_upper(make_kde_cbar)
pg.fig.set_size_inches(15,10)
pg.fig.savefig(f"{det}_fixed_astroph.png", dpi =300)
pg.fig.savefig(f"{det}_fixed_astroph.pdf", dpi =300)

In [ ]:
items = selected_keys_sets[0]
data = get_df(items)
print(np.unique(data.mass), len(data))
data.head(3)

In [ ]:
%%time
# data = sns.load_dataset('iris')
# plt.figure(figsize=(15,10))
def make_kde(*args, **kwargs):  
    #"cbar":True, 
#     opts ={"n_levels":5}
    sns.kdeplot(n_levels = 3, *args, cmap=next(make_kde.cmap_cycle), **kwargs)

def make_kde_cbar(*args, **kwargs):  
    #"cbar":True, 
#     opts ={"n_levels":5}
    sns.kdeplot(n_levels = 3, cbar = True, *args, cmap=next(make_kde.cmap_cycle), **kwargs)
kwargs ={"cbar":True, "n_levels":5}
make_kde.cmap_cycle = cycle(('Blues_r', 'Oranges_r', 'Greens_r'))

pg = sns.PairGrid(data, vars=('$\log(M_\chi)$', '$\log(\sigma)$'), hue='mass')
pg.map_diag(sns.kdeplot)#, color = ({)'b', 'g', 'r'})
pg.map_lower(make_kde)
# pg.map_upper(make_kde_cbar)
pg.fig.set_size_inches(15,10)
pg.fig.savefig(f"{det}_loose_astroph.png", dpi =300)
pg.fig.savefig(f"{det}_loose_astroph.pdf", dpi =300)

In [ ]:
# np.shape(np.random.normal(1, 1, 10**6))


In [ ]:
from scipy.stats.mstats import mquantiles

In [ ]:
# x, y = np.random.multivariate_normal(mean, cov, 1000).T
with sns.axes_style("white"):
    sns.jointplot(x=test[0], y=test[1], kind="hex", color="k");

In [ ]:
f, ax = plt.subplots(figsize=(6, 6))
sns.kdeplot(test[0], test[1], ax=ax)
sns.rugplot(test[0], color="g", ax=ax)
sns.rugplot(test[1], vertical=True, ax=ax);

In [ ]:
test_df = pd.DataFrame()
test_df['x'] = test[0]
test_df['y'] = test[1]

In [ ]:
sns.jointplot(x="x", y="y", data=test_df, kind="kde");

In [ ]:
items = selected_keys

def plot_posterior(items):
    posteriors = [get_p_i(item) for item in items]
    test_df = pd.DataFrame()

    test_df['x'] = np.concatenate([p[0] for p in posteriors])
    test_df['y'] = np.concatenate([p[1] for p in posteriors])
    sns.jointplot(x="x", y="y", data=test_df, kind="kde",
                 ylim = [-46,-44],
                 xlim = [1,3]);
plot_posterior(items)

In [ ]:
items = selected_keys[:3]
def plot_posterior_per_item(items):
    for item in items:
        posteriors = get_p_i(item)
        test_df = pd.DataFrame()

        test_df['x'] = posteriors[0]
        test_df['y'] = posteriors[1]
        sns.jointplot(x="x", y="y", data=test_df, kind="kde",
                     ylim = [-46,-44],
                     xlim = [1,3]);
plot_posterior_per_item(items)

In [ ]:
# items = selected_keys[:3]

# # sns.jointplot(x="x", y="y", data=test_df, kind="kde",
# #                      ylim = [-46,-44],
# #                      xlim = [1,3]);
# def plot_posterior_per_item(items):
#     for j, item in enumerate(items):
#         posteriors = get_p_i(item)
#         test_df = pd.DataFrame()

#         test_df[f'x'] = posteriors[0]
#         test_df[f'y'] = posteriors[1]
#         if j ==0:
            
#             grid = sns.JointGrid(x="x", y="y", data=test_df)
#         grid = grid.plot_joint(
#             sns.jointplot(x="x", y="y", data=test_df, kind="kde",
#                      ylim = [-46,-44],
#                      xlim = [1,3]));
# plot_posterior_per_item(items)

In [ ]:
selected_keys

In [ ]:
# %time
# items = results.keys()
# def get_p_i(i):
#     return get_posterior(results[i]['samples'], results[i]['weights']).T[:2]
# posteriors = [get_p_i(item) for item in items]
# test_df = pd.DataFrame()

# test_df['x'] = np.concatenate([p[0] for p in posteriors])
# test_df['y'] = np.concatenate([p[1] for p in posteriors])
# sns.jointplot(x="x", y="y", data=test_df, kind="kde",
#              ylim = [-46,-44],
#              xlim = [1,3]);